<a href="https://colab.research.google.com/github/harenlin/PySpark-Learning/blob/main/Search_and_Filter_DataFrames_in_PySpark_HW.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Search and Filter DataFrames in PySpark HW

Now it's time to put what you've learn into action with a homework assignment!

In case you need it again, here is the link to the documentation for the full list available function in pyspark.sql.functions library:
http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#module-pyspark.sql.functions


### First set up your Spark Session!
Alright so first things first, let's start up our pyspark instance.

In [1]:
!pip install pyspark
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('SFDataframe').getOrCreate()
cores = spark._jsc.sc().getExecutorMemoryStatus().keySet().size()
print("You are working with", cores, "core(s)")
spark

     |████████████████████████████████| 212.4MB 68kB/s 
     |████████████████████████████████| 204kB 13.1MB/s 
  Created wheel for pyspark: filename=pyspark-3.1.2-py2.py3-none-any.whl size=212880768 sha256=d79ad2bb8fdeef54dea1ce363eb81e9742c78a01e09e343e2a37bf15d4a4369c
  Stored in directory: /root/.cache/pip/wheels/40/1b/2c/30f43be2627857ab80062bef1527c0128f7b4070b6b2d02139
Successfully built pyspark
You are working with 1 core(s)


## Read in the DataFrame for this Notebook

We will be continuing to use the fifa19.csv file for this notebook. Make sure that you are writting the correct path to the file. 

In [2]:
from google.colab import drive
drive.mount('/content/drive')

path = '/content/drive/My Drive/PySpark/Datasets/'
fifa = spark.read.csv(path + 'fifa19.csv', inferSchema=True, header=True)

Mounted at /content/drive


## About this dataframe

The **fifa19.csv** dataset includes a list of all the FIFA 2019 players and their attributes listed below: 

 - **General**: Age, Nationality, Overall, Potential, Club
 - **Metrics:** Value, Wage
 - **Player Descriptive:** Preferred Foot, International Reputation, Weak Foot, Skill Moves, Work Rate, Position, Jersey Number, Joined, Loaned From, Contract Valid Until, Height, Weight
 - **Possition:** LS, ST, RS, LW, LF, CF, RF, RW, LAM, CAM, RAM, LM, LCM, CM, RCM, RM, LWB, LDM, CDM, RDM, RWB, LB, LCB, CB, RCB, RB, 
 - **Other:** Crossing, Finishing, Heading, Accuracy, ShortPassing, Volleys, Dribbling, Curve, FKAccuracy, LongPassing, BallControl, Acceleration, SprintSpeed, Agility, Reactions, Balance, ShotPower, Jumping, Stamina, Strength, LongShots, Aggression, Interceptions, Positioning, Vision, Penalties, Composure, Marking, StandingTackle, SlidingTackle, GKDiving, GKHandling, GKKicking, GKPositioning, GKReflexes, and Release Clause.

**Source:** https://www.kaggle.com/karangadiya/fifa19

Use the .toPandas() method to view the first few lines of the dataset so we know what we are working with. 

In [3]:
fifa.limit(5).toPandas()

,_c0,ID,Name,Age,Photo,Nationality,Flag,Overall,Potential,Club,Club Logo,Value,Wage,Special,Preferred Foot,International Reputation,Weak Foot,Skill Moves,Work Rate,Body Type,Real Face,Position,Jersey Number,Joined,Loaned From,Contract Valid Until,Height,Weight,LS,ST,RS,LW,LF,CF,RF,RW,LAM,CAM,RAM,LM,...,LB,LCB,CB,RCB,RB,Crossing,Finishing,HeadingAccuracy,ShortPassing,Volleys,Dribbling,Curve,FKAccuracy,LongPassing,BallControl,Acceleration,SprintSpeed,Agility,Reactions,Balance,ShotPower,Jumping,Stamina,Strength,LongShots,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Marking,StandingTackle,SlidingTackle,GKDiving,GKHandling,GKKicking,GKPositioning,GKReflexes,Release Clause
0,0,158023,L. Messi,31,https://cdn.sofifa.org/players/4/19/158023.png,Argentina,https://cdn.sofifa.org/flags/52.png,94,94,FC Barcelona,https://cdn.sofifa.org/teams/2/light/241.png,€110.5M,€565K,2202,Left,5,4,4,Medium/ Medium,Messi,Yes,RF,10,"Jul 1, 2004",None,2021,5'7,159lbs,88+2,88+2,88+2,92+2,93+2,93+2,93+2,92+2,93+2,93+2,93+2,91+2,...,59+2,47+2,47+2,47+2,59+2,84,95,70,90,86,97,93,94,87,96,91,86,91,95,95,85,68,72,59,94,48,22,94,94,75,96,33,28,26,6,11,15,14,8,€226.5M
1,1,20801,Cristiano Ronaldo,33,https://cdn.sofifa.org/players/4/19/20801.png,Portugal,https://cdn.sofifa.org/flags/38.png,94,94,Juventus,https://cdn.sofifa.org/teams/2/light/45.png,€77M,€405K,2228,Right,5,4,5,High/ Low,C. Ronaldo,Yes,ST,7,"Jul 10, 2018",None,2022,6'2,183lbs,91+3,91+3,91+3,89+3,90+3,90+3,90+3,89+3,88+3,88+3,88+3,88+3,...,61+3,53+3,53+3,53+3,61+3,84,94,89,81,87,88,81,76,77,94,89,91,87,96,70,95,95,88,79,93,63,29,95,82,85,95,28,31,23,7,11,15,14,11,€127.1M
2,2,190871,Neymar Jr,26,https://cdn.sofifa.org/players/4/19/190871.png,Brazil,https://cdn.sofifa.org/flags/54.png,92,93,Paris Saint-Germain,https://cdn.sofifa.org/teams/2/light/73.png,€118.5M,€290K,2143,Right,5,5,5,High/ Medium,Neymar,Yes,LW,10,"Aug 3, 2017",None,2022,5'9,150lbs,84+3,84+3,84+3,89+3,89+3,89+3,89+3,89+3,89+3,89+3,89+3,88+3,...,60+3,47+3,47+3,47+3,60+3,79,87,62,84,84,96,88,87,78,95,94,90,96,94,84,80,61,81,49,82,56,36,89,87,81,94,27,24,33,9,9,15,15,11,€228.1M
3,3,193080,De Gea,27,https://cdn.sofifa.org/players/4/19/193080.png,Spain,https://cdn.sofifa.org/flags/45.png,91,93,Manchester United,https://cdn.sofifa.org/teams/2/light/11.png,€72M,€260K,1471,Right,4,3,1,Medium/ Medium,Lean,Yes,GK,1,"Jul 1, 2011",None,2020,6'4,168lbs,None,None,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,17,13,21,50,13,18,21,19,51,42,57,58,60,90,43,31,67,43,64,12,38,30,12,68,40,68,15,21,13,90,85,87,88,94,€138.6M
4,4,192985,K. De Bruyne,27,https://cdn.sofifa.org/players/4/19/192985.png,Belgium,https://cdn.sofifa.org/flags/7.png,91,92,Manchester City,https://cdn.sofifa.org/teams/2/light/10.png,€102M,€355K,2281,Right,4,5,4,High/ High,Normal,Yes,RCM,7,"Aug 30, 2015",None,2023,5'11,154lbs,82+3,82+3,82+3,87+3,87+3,87+3,87+3,87+3,88+3,88+3,88+3,88+3,...,73+3,66+3,66+3,66+3,73+3,93,82,55,92,82,86,85,83,91,91,78,76,79,91,77,91,63,90,75,91,76,61,87,94,79,88,68,58,51,15,13,5,10,13,€196.4M


Now print the schema of the dataset so we can see the data types of all the varaibles. 

In [4]:
print(fifa.printSchema())

root
 |-- _c0: integer (nullable = true)
 |-- ID: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Photo: string (nullable = true)
 |-- Nationality: string (nullable = true)
 |-- Flag: string (nullable = true)
 |-- Overall: integer (nullable = true)
 |-- Potential: integer (nullable = true)
 |-- Club: string (nullable = true)
 |-- Club Logo: string (nullable = true)
 |-- Value: string (nullable = true)
 |-- Wage: string (nullable = true)
 |-- Special: integer (nullable = true)
 |-- Preferred Foot: string (nullable = true)
 |-- International Reputation: integer (nullable = true)
 |-- Weak Foot: integer (nullable = true)
 |-- Skill Moves: integer (nullable = true)
 |-- Work Rate: string (nullable = true)
 |-- Body Type: string (nullable = true)
 |-- Real Face: string (nullable = true)
 |-- Position: string (nullable = true)
 |-- Jersey Number: integer (nullable = true)
 |-- Joined: string (nullable = true)
 |-- Loaned From: string (nu

## Now let's get started!

### First things first..... import the pyspark sql functions library

Since we know we will be using it a lot.

In [5]:
from pyspark.sql.functions import *

### 1. Select the Name and Position of each player in the dataframe

In [9]:
fifa.select(['Name', 'Position']).show(5, False)

+-----------------+--------+
|Name             |Position|
+-----------------+--------+
|L. Messi         |RF      |
|Cristiano Ronaldo|ST      |
|Neymar Jr        |LW      |
|De Gea           |GK      |
|K. De Bruyne     |RCM     |
+-----------------+--------+
only showing top 5 rows



### 1.1 Display the same results from above sorted by the players names

In [10]:
fifa.select(['Name','Position']).orderBy('Name').show(5, False)

+-------------+--------+
|Name         |Position|
+-------------+--------+
|A. Abang     |ST      |
|A. Abdellaoui|LB      |
|A. Abdennour |CB      |
|A. Abdi      |CM      |
|A. Abdu Jaber|ST      |
+-------------+--------+
only showing top 5 rows



### 2. Select only the players who belong to a club begining with FC

In [11]:
# method 1
fifa.select(['Name', 'Club']).where(fifa.Club.like("FC%")).show(5, False)

+---------------+-----------------+
|Name           |Club             |
+---------------+-----------------+
|L. Messi       |FC Barcelona     |
|L. Suárez      |FC Barcelona     |
|R. Lewandowski |FC Bayern München|
|M. ter Stegen  |FC Barcelona     |
|Sergio Busquets|FC Barcelona     |
+---------------+-----------------+
only showing top 5 rows



In [13]:
fifa.select(['Name', 'Club']).where(fifa.Club.startswith("FC")).show(5, False)

+---------------+-----------------+
|Name           |Club             |
+---------------+-----------------+
|L. Messi       |FC Barcelona     |
|L. Suárez      |FC Barcelona     |
|R. Lewandowski |FC Bayern München|
|M. ter Stegen  |FC Barcelona     |
|Sergio Busquets|FC Barcelona     |
+---------------+-----------------+
only showing top 5 rows



### 3. Who is the oldest player in the dataset and how old are they?

Display only the name and age of the oldest player.

In [18]:
fifa.select(['Name', 'Age']).sort(fifa["Age"].desc()).show(5)

+-------------+---+
|         Name|Age|
+-------------+---+
|     O. Pérez| 45|
|K. Pilkington| 44|
|    T. Warner| 44|
|  S. Narazaki| 42|
|     M. Tyler| 41|
+-------------+---+
only showing top 5 rows



In [19]:
fifa.select(['Name', 'Age']).sort(desc("Age")).show(5)

+-------------+---+
|         Name|Age|
+-------------+---+
|     O. Pérez| 45|
|K. Pilkington| 44|
|    T. Warner| 44|
|  S. Narazaki| 42|
|     M. Tyler| 41|
+-------------+---+
only showing top 5 rows



In [16]:
fifa.select(['Name', 'Age']).orderBy(fifa["Age"].desc()).show(5,False)

+-------------+---+
|Name         |Age|
+-------------+---+
|O. Pérez     |45 |
|K. Pilkington|44 |
|T. Warner    |44 |
|S. Narazaki  |42 |
|M. Tyler     |41 |
+-------------+---+
only showing top 5 rows



### 4. Select only the following players from the dataframe:

 - L. Messi
 - Cristiano Ronaldo

In [20]:
fifa[fifa.Name.isin("L. Messi", "Cristiano Ronaldo")].show(5, False)

+---+------+-----------------+---+----------------------------------------------+-----------+-----------------------------------+-------+---------+------------+--------------------------------------------+-------+-----+-------+--------------+------------------------+---------+-----------+--------------+----------+---------+--------+-------------+------------+-----------+--------------------+------+------+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+--------+---------+---------------+------------+-------+---------+-----+----------+-----------+-----------+------------+-----------+-------+---------+-------+---------+-------+-------+--------+---------+----------+-------------+-----------+------+---------+---------+-------+--------------+-------------+--------+----------+---------+-------------+----------+--------------+
|_c0|ID    |Name             |Age|Photo                                         |Nation

In [22]:
fifa[fifa.Name.isin("L. Messi", "Cristiano Ronaldo")].select(['Name', 'Age']).show(5, False)

+-----------------+---+
|Name             |Age|
+-----------------+---+
|L. Messi         |31 |
|Cristiano Ronaldo|33 |
+-----------------+---+



### 5. Can you select the first character from the Release Clause variable which indicates the currency used?

In [24]:
fifa.select("Release Clause", fifa["Release Clause"].substr(1,1).alias("currency")).show(5,False)

+--------------+--------+
|Release Clause|currency|
+--------------+--------+
|€226.5M       |€       |
|€127.1M       |€       |
|€228.1M       |€       |
|€138.6M       |€       |
|€196.4M       |€       |
+--------------+--------+
only showing top 5 rows



### 6. Can you select only the players who are over the age of 40?

In [25]:
fifa.filter("Age>40").show(5)

+-----+------+-----------+---+--------------------+-----------+--------------------+-------+---------+--------------------+--------------------+-----+----+-------+--------------+------------------------+---------+-----------+--------------+---------+---------+--------+-------------+-----------+-----------+--------------------+------+------+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+--------+---------+---------------+------------+-------+---------+-----+----------+-----------+-----------+------------+-----------+-------+---------+-------+---------+-------+-------+--------+---------+----------+-------------+-----------+------+---------+---------+-------+--------------+-------------+--------+----------+---------+-------------+----------+--------------+
|  _c0|    ID|       Name|Age|               Photo|Nationality|                Flag|Overall|Potential|                Club|           Club Logo|Value|Wag

In [26]:
fifa.filter("Age>40").select(['Name','Age']).show(5)

+-----------+---+
|       Name|Age|
+-----------+---+
|  J. Villar| 41|
|   B. Nivet| 41|
|   O. Pérez| 45|
|   C. Muñoz| 41|
|S. Narazaki| 42|
+-----------+---+
only showing top 5 rows



### That's is for now... Great Job!